In [2]:
import os
os.environ["KMP_DUPLICATE_LIB_OK"]="TRUE"
#os.environ['CUDA_LAUNCH_BLOCKING'] = str(1)
#os.environ["TORCH_USE_CUDA_DSA"]= str(0)
device = "cuda"

In [3]:
import copy
import torch
import torch.nn as nn
from torch.nn import functional as F
from torch.utils.data import Dataset
import torch.optim as optim
from torch.autograd import Variable
from tqdm import tqdm
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import time
import math
from collections import OrderedDict
import random
from torchsummary import summary

In [3]:
X_train = torch.randn([425833, 300, 18])
Y_train = torch.ones([425833, 1]) * 0.5
X_val = torch.randn([47314, 300, 18])
Y_val = torch.ones([47314, 1]) * 0.5
X_test = torch.randn([118286, 300, 18])
Y_test = torch.ones([118286, 1]) * 0.5

In [12]:
class LSTM_CNN_Spatial(nn.Module):
    def __init__(self, num_classes, batch_size, T, C, input_size, hidden_size,
                 num_layers, spatial_num=8, drop_out=0.5):
        super(LSTM_CNN_Spatial, self).__init__()

        self.N = batch_size
        self.T = T
        self.C = C
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.pool = 4
        self.seq_len = self.T // self.input_size
        self.fc_in = spatial_num * self.hidden_size // self.pool
        
        self._lstm = nn.LSTM(self.input_size, self.hidden_size, 
                            self.num_layers, batch_first=True)
        self.lstm = nn.ModuleList([self._lstm for i in range(self.C)])
        
        self.block_1 = nn.Sequential(
            nn.Conv2d(1, spatial_num, (self.C, 1)),
            nn.BatchNorm2d(spatial_num),
            nn.ELU(),
            nn.AvgPool2d((1, self.pool)),
            nn.Dropout(drop_out)
        )
        
        self.fc = nn.Linear(self.fc_in , num_classes)
        
    def forward(self, x):
        # input shape of x: (N, 1, C, T)
        self.N = x.shape[0]
        x = x.reshape(self.N, self.C, self.seq_len, self.input_size)
        _x = None
        for index, lstm in enumerate(self.lstm):
            lstm_out, _ = lstm(x[:, index, :, :], None)
            tmp = lstm_out[:, -1, :]
            tmp = tmp.unsqueeze(0)
            if _x is None:
                _x = tmp
            else:
                _x = torch.cat((_x, tmp), dim=0)
        
        # (C, N, H) ===> (N, 1, C, H)   H: hidden_size
        x = _x.permute(1, 0, 2).unsqueeze(1)
        x = self.block_1(x)
        
        x = x.view(x.size(0), -1)
        logits = self.fc(x)
        probas = F.softmax(logits, dim=1)
        return probas  

In [14]:
if __name__ == '__main__':
    # model test
    DEVICE = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
    x = torch.randn((4, 1, 18, 300))
    x = x.to(DEVICE)
    # num_classes = 4, batch_size = 4, T = 256, C = 64, input_size = 16, hidden_size = 16, num_layers = 2, spatial_num=8, drop_out=0.5
    model = LSTM_CNN_Spatial(4, 4, 300, 18, 15, 160, 20)
    model = model.to(DEVICE)
    y = model(x)
    print(y.data)

tensor([[0.2316, 0.2906, 0.1707, 0.3071],
        [0.1876, 0.2736, 0.2095, 0.3293],
        [0.1901, 0.3396, 0.1666, 0.3038],
        [0.1804, 0.2422, 0.2206, 0.3568]], device='cuda:0')
